In [198]:
import pandas as pd

In [199]:
df_mimic = pd.read_csv("../data/raw/mimic_iii/DIAGNOSES_ICD.csv")
df_mimic = df_mimic.dropna(subset=["ICD9_CODE"], axis=0)
df_mimic["ICD9_CODE"] = df_mimic["ICD9_CODE"].str[:3]
df_mimic.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,403
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,582
3,1300,109,172335,4.0,585
4,1301,109,172335,5.0,425


In [200]:
ICD9_SPECIALTY_DICT = {
    "Certain Conditions Originating In The Perinatal Period": "Obstetrics & Gynaecology",
    "Complications Of Pregnancy, Childbirth, And The Puerperium": "Obstetrics & Gynaecology",
    "Congenital Anomalies": "Primary Care",
    "Diseases Of The Blood And Blood-Forming Organs": "Hematology",
    "Diseases Of The Circulatory System": "Cardiothoracic & Vascular",
    "Diseases Of The Digestive System": "Gastroenterology",
    "Diseases Of The Genitourinary System": "Urology",
    "Diseases Of The Musculoskeletal System And Connective Tissue": "Orthopedic surgery",
    "Diseases Of The Nervous System And Sense Organs": "Neurology",
    "Diseases Of The Respiratory System": "Pulmonology",
    "Diseases Of The Skin And Subcutaneous Tissue": "Dermatology",
    "Endocrine, Nutritional And Metabolic Diseases, And Immunity Disorders": "Endocrinology",
    "Infectious And Parasitic Diseases": "Infectious Disease Specialty",
    "Injury And Poisoning": "Emergency Department",
    "Mental Disorders": "Psychiatry",
    "Neoplasms": "Oncology",
    "Supplementary Classification Of External Causes Of Injury And Poisoning": "Emergency Department",
    "Supplementary Classification Of Factors Influencing Health Status And Contact With Health Services": "Internal Medicine Department",
    "Symptoms, Signs, And Ill-Defined Conditions": "Internal Medicine Department",
}


In [209]:
df_9 = pd.read_csv("../data/interim/icd9_codes_and_des.csv")
df_9["category_codes"] = df_9["category_codes"].str.rjust(3, "0")
df_9.loc[df_9["category_codes"].str.startswith("E"), "category_codes"] = df_9["category_codes"].str[:3]
df_9 = df_9.drop_duplicates(subset=["category_codes", "chapter_codes_des"], keep="first")
df_9.head()

,category_codes,category_codes_des,block_codes,block_codes_des,chapter_codes,chapter_codes_des
0,001,Cholera,001-009,Intestinal Infectious Diseases,001-139,Infectious And Parasitic Diseases
1,002,Typhoid and paratyphoid fevers,001-009,Intestinal Infectious Diseases,001-139,Infectious And Parasitic Diseases
2,003,Other salmonella infections,001-009,Intestinal Infectious Diseases,001-139,Infectious And Parasitic Diseases
3,004,Shigellosis,001-009,Intestinal Infectious Diseases,001-139,Infectious And Parasitic Diseases
4,005,Other food poisoning (bacterial),001-009,Intestinal Infectious Diseases,001-139,Infectious And Parasitic Diseases


In [210]:
df = pd.merge(df_mimic, df_9, left_on="ICD9_CODE", right_on="category_codes", how="left")
print(df.shape)
print(df.ROW_ID.nunique())
df["specialty"] = df["chapter_codes_des"].map(ICD9_SPECIALTY_DICT)
df = df.drop(columns=["category_codes", "category_codes_des", "block_codes", "block_codes_des", "chapter_codes", "chapter_codes_des"])


(651000, 11)
651000


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,specialty
0,1297,109,172335,1.0,403,Cardiothoracic & Vascular
1,1298,109,172335,2.0,486,Pulmonology
2,1299,109,172335,3.0,582,Urology
3,1300,109,172335,4.0,585,Urology
4,1301,109,172335,5.0,425,Cardiothoracic & Vascular


In [211]:
df.head(15)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,specialty
0,1297,109,172335,1.0,403,Cardiothoracic & Vascular
1,1298,109,172335,2.0,486,Pulmonology
2,1299,109,172335,3.0,582,Urology
3,1300,109,172335,4.0,585,Urology
4,1301,109,172335,5.0,425,Cardiothoracic & Vascular
5,1302,109,172335,6.0,276,Endocrinology
6,1303,109,172335,7.0,710,Orthopedic surgery
7,1304,109,172335,8.0,276,Endocrinology
8,1305,109,172335,9.0,724,Orthopedic surgery
9,1306,109,172335,10.0,458,Cardiothoracic & Vascular


In [215]:
df.sort_values("SEQ_NUM", ascending=True)

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,specialty
0,1297,109,172335,1.0,403,Cardiothoracic & Vascular
111738,105624,9413,176956,1.0,865,Emergency Department
111730,105616,9412,148755,1.0,996,Emergency Department
519027,523522,71349,102891,1.0,038,Infectious Disease Specialty
335280,328441,28882,166021,1.0,441,Cardiothoracic & Vascular
...,...,...,...,...,...,...
507288,497762,65837,184409,39.0,V44,Internal Medicine Department
648723,637430,96958,102063,39.0,V49,Internal Medicine Department
547428,547312,76736,113085,39.0,275,Endocrinology
451920,435135,52269,113427,39.0,427,Cardiothoracic & Vascular


In [220]:
test = df.drop_duplicates(subset=["SUBJECT_ID", "HADM_ID"], keep="first")
test.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE,specialty
0,1297,109,172335,1.0,403,Cardiothoracic & Vascular
14,1311,109,173633,1.0,403,Cardiothoracic & Vascular
28,1488,112,174105,1.0,531,Gastroenterology
33,1493,113,109976,1.0,191,Oncology
36,1496,114,178393,1.0,414,Cardiothoracic & Vascular


In [241]:
import pandas as pd
import os

In [244]:
DIAGNOSES_NOTEEVENTS_CSV = os.path.join(
    "..", "data", "processed", "mimic_iii", "diagnoses_noteevents.csv"
)
df = pd.read_csv(DIAGNOSES_NOTEEVENTS_CSV)

df.shape

(58929, 2)